## Excel to report

In [ ]:
# read file path
import os
import datetime
import pandas as pd

today = datetime.date.today()
year = today.year-1

path = os.getcwd() + "\data\sales_data" # change to my floder name

xlxs_file_lists = []

for root,dirs,files in os.walk(path):
      for name in files:
        file_path = os.path.join(root,name)
        #print(file_path.split("\\")[-2])
        if file_path.split("\\")[-2] == str(year): #change filter
            xlxs_file_lists.append(file_path)
xlxs_file_lists

In [ ]:
#export to dataframe
dfs = []
for f in xlxs_file_lists:
  #print(f.split("\\")[-1])
  df = pd.read_excel(f)
  df["file_name"] = f.split("\\")[-1]
  df["year"] = file_path.split("\\")[-2]
  df["amount_10x"] = df["amount"]*10
  dfs.append(df)
dfs

In [ ]:
dfs_summary = pd.concat(dfs)

In [ ]:
dfs_summary

In [ ]:
dfs_summary.info()

In [ ]:
dfs_summary.nunique()

In [ ]:
dfs_summary["plan"].unique()

In [ ]:
dfs_summary.describe()

In [ ]:
dfs_summary.head(5)

In [ ]:
dfs_summary.tail(5)

In [ ]:
dfs_summary

In [ ]:
pivot = pd.pivot_table(dfs_summary,index="transaction_date",columns="store",values="amount",aggfunc="sum")
pivot

In [ ]:
summary_monthly = pivot.resample("M").sum()
summary_monthly

In [ ]:
#!pip install matplotlib
import matplotlib

In [ ]:
fig = summary_monthly.plot(kind="bar",figsize=(20,12),fontsize=26,title="daily summary amount").get_figure()

xlwings for excel

In [ ]:
#!pip install xlwings

In [ ]:
import xlwings as xw

In [ ]:
import xlwings as xw

import datetime
now = datetime.datetime.now()
date_file_name = f'{str(now.date())}_{str(now.time()).split(".")[0].replace(":","_")}'


template = xw.Book(r"D:\My Documents\Desktop\python_office\src\data\sale_template.xlsx")

app = xw.apps.active
sheet = template.sheets["summary"]
sheet["A1"].value = summary_monthly

pivote = template.sheets["pivot"]
pivote["A1"].value = pivot

#add picture
sheet_report = template.sheets["report"]
sheet_report["A1"].value = "Summary by month"
sheet_report['A1'].font.size = 24
sheet_report["A1"].api.Font.Bold = True
plot= sheet_report.pictures.add(fig,top=sheet["A3"].top,left=sheet["A3"].left)
plot.width = plot.width*0.8
plot.height = plot.height*0.8

template.save(f"export\summary_sale_report_{date_file_name}.xlsx")
template.close()
app.kill()

#### script node-red
conda activate python_office && python "D:\My Documents\Desktop\python_office\src\sale_report_script.py"

#### install pm2
- npm install -g pm2 
- pm2 status
- pm2 start C:\Users\<User>\AppData\Roaming\npm\node_modules\node-red\red.js
- pm2 start C:\Users\lbj6639\AppData\Roaming\npm\node_modules\node-red\red.js
- npm install pm2-windows-startup -g
- pm2-startup install
- pm2 save

#### send email

In [ ]:
#!pip install email

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from email.mime.text import MIMEText

from datetime import date

today = date.today()

send_from = 'devops.mic@gmail.com'
#send_to = 'devops.mic@gmail.com'
send_to = ['suraphop.b@minebea.co.th','devops.mic@gmail.com']
subject = f"sale summary report {today}"
text = f'''<html>
        <head><b>Dear sir</b></head>
        <body>
            <div>
              I would like to report summary sale result with a attach file as below
            </div>
            <p>Suraphop Bunsawat<br>MIC division<br>Tel: 2236</p><br>
            <div>
            <i>a attactfile</i>
            </div>
        </body>
    </html>'''

files = f"export\summary_sale_report_{date_file_name}.xlsx"
file_name_1 = f"summary_sale_report_{date_file_name}_1.xlsx"
file_name_2 = f"summary_sale_report_{date_file_name}_2.xlsx"
file_name_3 = f"summary_sale_report_{date_file_name}_3.xlsx"

username = 'devops.mic@gmail.com'
password = 'knhxhmritpsyqbjc'

msg = MIMEMultipart()
msg['Subject'] = subject 
msg['From'] = send_from
msg['To'] = ', '.join(send_to)

msg.attach(MIMEText(text,'html'))

part = MIMEBase('application', "octet-stream")
with open(files, 'rb') as f:
    file = f.read()
part.set_payload(file)
encoders.encode_base64(part)    
part.add_header('Content-Disposition', f'attachment; filename="{file_name_1}"')
msg.attach(part)

part2 = MIMEBase('application', "octet-stream")
with open(files, 'rb') as f:
    file = f.read()
part2.set_payload(file)
encoders.encode_base64(part2)    
part2.add_header('Content-Disposition', f'attachment; filename="{file_name_2}"')
msg.attach(part2)

part3 = MIMEBase('application', "octet-stream")
with open(files, 'rb') as f:
    file = f.read()
part3.set_payload(file)
encoders.encode_base64(part3)    
part3.add_header('Content-Disposition', f'attachment; filename="{file_name_3}"')
msg.attach(part3)

server = smtplib.SMTP("smtp.gmail.com", 587)
server.ehlo()
server.starttls()
server.login(username, password)
server.sendmail(send_from,send_to, msg.as_string())

#### Line notify

In [ ]:
#!pip install requests

In [44]:
import requests
  
def line_notify(token,msg):
    try:
        url = 'https://notify-api.line.me/api/notify'
        headers = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer '+token}
        r = requests.post(url, headers=headers, data = {'message':msg})
        return r.text
    except Exception as e:
        return e
msg = f'''ส่งรายงานเรื่อง {subject} ไปที่ 
{send_to[0]} และ {send_to[1]}'''
line_notify("4ZiC8AcMDJANba4a8pLAEcJkb4v07kpVW8DzaGKWtBU",msg)

'{"status":200,"message":"ok"}'